In [ ]:
from requests import Request, Session
import json
import numpy as np
import pandas as pd

In [ ]:
import time

In [ ]:
# url = 'https://cloud.iexapis.com/stable/stock/AAPL/book'
url_book = 'https://cloud.iexapis.com/stable/deep/book'
url_trade = 'https://cloud.iexapis.com/stable/deep/trades'
# url = 'https://sandbox.iexapis.com/stable/deep/book'

In [ ]:
parameters_book = {
#     'token': 'Tsk_83f2c0eee6bb4600b8d8f9936a5769f2',
    'token': 'sk_b8918e562940417ca9b6615a914b75bd',
    'symbols': "AAPL"
}
parameters_trade = {
    'token': 'sk_b8918e562940417ca9b6615a914b75bd',
    'symbols': 'AAPL',
    'last': 500
}
headers = {
    'Accepts': 'application/json'
}

In [ ]:
session = Session()
session.headers.update(headers)

# Function

In [ ]:
def crawl_book():
    columns = ['price','size','timestamp']
    bids = pd.DataFrame(columns=columns)
    asks = pd.DataFrame(columns=columns)
    response = session.get(url_book, params=parameters_book)
    if not response.status_code == 200:
        print(response.text)
        return bids, asks
    data = json.loads(response.text)
    if not 'AAPL' in data:
        return bids, asks
    else:
        data = data.get('AAPL')
    if 'bids' in data and 'asks' in data:
        bids = pd.DataFrame(data.get('bids'))
        asks = pd.DataFrame(data.get('asks')) 
    return bids, asks

In [ ]:
def crawl_trade():
    columns = ['price','size','timestamp']
    trades = pd.DataFrame(columns=columns)
    response = session.get(url_trade, params=parameters_trade)
    if not response.status_code == 200:
        print(response.text)
        return trades
    data = json.loads(response.text)
    if not 'AAPL' in data:
        return trades
    else:
        data = pd.DataFrame(data.get('AAPL'))
    if not data.empty:
        trades = data[['price','size','timestamp']]
    return trades

# Collect data

In [ ]:
total_trades = pd.read_csv('trades.csv')

## New data

In [70]:
i = 0
bids = pd.DataFrame(columns=['price','size','timestamp'])
asks = pd.DataFrame(columns=['price','size','timestamp'])
while i >= 0 and i < 50:
    new_bids, new_asks = crawl_book()
    trades = pd.DataFrame(columns=['price','size','timestamp'])
    new_trades = crawl_trade()
    trades = trades.append(new_trades)
    cur_time = round(time.time())
    if not new_bids.empty and not new_bids.equals(bids):
        new_bids.to_csv('bids/{0}.csv'.format(cur_time), index=False)
        bids = new_bids
    if not new_asks.empty and not new_asks.equals(asks):
        new_asks.to_csv('asks/{0}.csv'.format(cur_time), index=False)
        asks = new_asks
    i = i + 1
    if i % 10 == 0: # mod 100
        print(trades.head(3))
        total_trades = total_trades.append(trades)
        total_trades.to_csv('trades.csv', index=False)
    time.sleep(1)

    price  size      timestamp
0  124.94    50  1614186195933
1  124.94  1000  1614186195022
2  124.94   200  1614186195022
    price size      timestamp
0  124.97  300  1614186211349
1  124.96  100  1614186211253
2  124.97    2  1614186210650
    price size      timestamp
0  124.96  109  1614186227229
1  124.97  300  1614186222610
2  124.98   24  1614186220984
    price size      timestamp
0  124.98    2  1614186242378
1  124.99  100  1614186240083
2  124.99  100  1614186240043
     price size      timestamp
0  124.930  100  1614186258889
1  124.945   78  1614186256154
2  124.910  100  1614186252047
